In [3]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.neighbors import LocalOutlierFactor
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, StratifiedKFold
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from collections import defaultdict
import itertools
import pickle

In [2]:
df = pd.read_csv("../data/run-over-dataset.csv")

cols_to_drop = ['VERBALE', 'DATA']
X = df.drop(columns=cols_to_drop)

X['ALTEZZA'] = [int(float(h.replace(',', '.'))*100) for h in X['ALTEZZA']]
X['PESO'] = [int(float(str(h).replace(',', '.'))) for h in X['PESO']]
X['BMI'] = [float(str(h).replace(',', '.')) for h in X['BMI']]

num_unique_values = X.nunique()
constant_columns = num_unique_values[num_unique_values == 1].index.tolist()

X = X.drop(columns=constant_columns)
X = X.T.drop_duplicates().T
X.head

<bound method NDFrame.head of      SESSO  ANNI  PESO  ALTEZZA        BMI  Mezzo  Testa:Neurocranio  \
0      0.0  81.0  84.0    175.0  27.428571    0.0                1.0   
1      1.0  69.0  69.0    162.0  26.291724    1.0                4.0   
2      1.0  71.0  67.0    155.0  27.887617    1.0                2.0   
3      1.0  54.0  60.0    159.0  23.733238    1.0                4.0   
4      1.0  78.0  69.0    167.0  24.740937    1.0                2.0   
..     ...   ...   ...      ...        ...    ...                ...   
125    1.0  82.0  80.0    162.0  30.559366    0.0                1.0   
126    1.0  70.0  75.0    157.0  30.670615    0.0                2.0   
127    0.0  45.0  45.0    177.0  14.363689    1.0                0.0   
128    0.0  54.0  89.0    168.0  31.746032    0.0                0.0   
129    1.0  86.0  50.0    155.0  20.811655    0.0                1.0   

     Testa:Splancnocranio  Testa:Telencefalo  Testa:Cervelletto  ...  \
0                     0.0        

In [8]:
def nested_cv_accuracy(X):
    n_neighborss = [5, 10, 15]
    contaminations = [0.01, 0.05, 0.1, 0.3, 0.5]
    metrics = ['euclidean', 'minkowski', 'manhattan']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_accuracy = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(n_neighborss, contaminations, metrics):
                    clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    accuracy = accuracy_score(true_values, pred_values)
                    if accuracy > best_accuracy:
                        best_accuracy = accuracy
                        best_params['n_neighbors'] = params[0]
                        best_params['contamination'] = params[1]
                        best_params['metric'] = params[2]

            clf = LocalOutlierFactor(**best_params, novelty=True)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'accuracy on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [9]:
results = nested_cv_accuracy(X)
results

{'algorythm': 'LocalOutlierFactor',
 'best n_neighbors': 10,
 'best contamination': 0.5,
 'best metric': 'manhattan',
 'method used for model selection': 'accuracy on nested cv',
 'accuracy mean': 60.153846153846146,
 'accuracy std': 8.694689939042947,
 'precision mean': 70.76915306915308,
 'precision  std': 13.875689928478193,
 'recall mean': 45.57142857142857,
 'recall std': 13.842407482008293,
 'data set': 'no const, no equals'}

In [10]:
scores_df = pd.DataFrame(results, index=[0])
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,10,0.5,manhattan,accuracy on nested cv,60.153846,8.69469,70.769153,13.87569,45.571429,13.842407,"no const, no equals"


In [12]:
def nested_cv_f1(X):
    n_neighborss = [5, 10, 15]
    contaminations = [0.01, 0.05, 0.1, 0.3, 0.5]
    metrics = ['euclidean', 'minkowski', 'manhattan']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=seed)

            best_f1 = 0
            for trainval_idx, valid_idx in inner_cv.split(X_train, y_train):
                X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
                y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

                idxs_neg = np.where(y_trainval == 1)[0]
                X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
                y_valid = np.append(y_valid, y_trainval[idxs_neg])

                X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
                y_trainval = np.delete(y_trainval, idxs_neg)

                for params in itertools.product(n_neighborss, contaminations, metrics):
                    clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                    clf.fit(X_trainval)

                    pred_values = clf.predict(X_valid)
                    true_values = [1 if y == 0 else -1 for y in y_valid]

                    f1 = f1_score(true_values, pred_values)
                    if f1 > best_f1:
                        best_f1 = f1
                        best_params['n_neighbors'] = params[0]
                        best_params['contamination'] = params[1]
                        best_params['metric'] = params[2]

            clf = LocalOutlierFactor(**best_params, novelty=True)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'f1 on nested cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [13]:
results = nested_cv_f1(X)

In [14]:
def add_record(df, record):
    new_record = pd.DataFrame(record, index=[0])
    df = pd.concat([df, new_record], ignore_index=True)
    return df  

In [15]:
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,10,0.5,manhattan,accuracy on nested cv,60.153846,8.694690,70.769153,13.875690,45.571429,13.842407,"no const, no equals"
1,LocalOutlierFactor,10,0.3,euclidean,f1 on nested cv,66.769231,9.093856,70.554775,8.376199,65.857143,16.554795,"no const, no equals"


In [16]:
def outer_cv_inner_holdout_accuracy(X):
    n_neighborss = [5, 10, 15]
    contaminations = [0.01, 0.05, 0.1, 0.3, 0.5]
    metrics = ['euclidean', 'minkowski', 'manhattan']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
            best_accuracy = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_neighborss, contaminations, metrics):
                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['n_neighbors'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['metric'] = params[2]

            clf = LocalOutlierFactor(**best_params, novelty=True)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'accuracy on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [17]:
results = outer_cv_inner_holdout_accuracy(X)

In [18]:
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,10,0.5,manhattan,accuracy on nested cv,60.153846,8.694690,70.769153,13.875690,45.571429,13.842407,"no const, no equals"
1,LocalOutlierFactor,10,0.3,euclidean,f1 on nested cv,66.769231,9.093856,70.554775,8.376199,65.857143,16.554795,"no const, no equals"
2,LocalOutlierFactor,5,0.5,manhattan,accuracy on outer cv inner holdout,62.000000,10.648777,71.489466,14.922015,49.285714,14.586127,"no const, no equals"


In [19]:
def outer_cv_inner_holdout_f1(X):
    n_neighborss = [5, 10, 15]
    contaminations = [0.01, 0.05, 0.1, 0.3, 0.5]
    metrics = ['euclidean', 'minkowski', 'manhattan']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        outer_cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)

        for train_idx, test_idx in outer_cv.split(X, y):
            X_train, X_test = X[train_idx], X[test_idx]
            y_train, y_test = y[train_idx], y[test_idx]

            X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)
            best_f1 = 0

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_neighborss, contaminations, metrics):
                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['n_neighbors'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['metric'] = params[2]

            clf = LocalOutlierFactor(**best_params, novelty=True)

            idxs_neg = np.where(y_train == 1)[0]
            X_train = np.delete(X_train, idxs_neg, axis=0)

            clf.fit(X_train)

            pred_values = clf.predict(X_test)
            true_values = [1 if y == 0 else -1 for y in y_test]

            accuracy_scores.append(accuracy_score(true_values, pred_values))
            precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
            recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'f1 on outer cv inner holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [20]:
results = outer_cv_inner_holdout_f1(X)

In [21]:
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,10,0.5,manhattan,accuracy on nested cv,60.153846,8.694690,70.769153,13.875690,45.571429,13.842407,"no const, no equals"
1,LocalOutlierFactor,10,0.3,euclidean,f1 on nested cv,66.769231,9.093856,70.554775,8.376199,65.857143,16.554795,"no const, no equals"
2,LocalOutlierFactor,5,0.5,manhattan,accuracy on outer cv inner holdout,62.000000,10.648777,71.489466,14.922015,49.285714,14.586127,"no const, no equals"
3,LocalOutlierFactor,5,0.3,manhattan,f1 on outer cv inner holdout,66.153846,9.421114,70.392742,10.398666,66.285714,21.094539,"no const, no equals"


In [22]:
def outer_holdout_inner_cv_accuracy(X):
    n_neighborss = [5, 10, 15]
    contaminations = [0.01, 0.05, 0.1, 0.3, 0.5]
    metrics = ['euclidean', 'minkowski', 'manhattan']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_accuracy = 0
        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_neighborss, contaminations, metrics):
                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                accuracy = accuracy_score(true_values, pred_values)
                if accuracy > best_accuracy:
                    best_accuracy = accuracy
                    best_params['n_neighbors'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['metric'] = params[2]

        clf = LocalOutlierFactor(**best_params, novelty=True)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'accuracy on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [23]:
results = outer_holdout_inner_cv_accuracy(X)

In [24]:
scores_df = add_record(scores_df, results)

In [25]:
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,10,0.5,manhattan,accuracy on nested cv,60.153846,8.694690,70.769153,13.875690,45.571429,13.842407,"no const, no equals"
1,LocalOutlierFactor,10,0.3,euclidean,f1 on nested cv,66.769231,9.093856,70.554775,8.376199,65.857143,16.554795,"no const, no equals"
2,LocalOutlierFactor,5,0.5,manhattan,accuracy on outer cv inner holdout,62.000000,10.648777,71.489466,14.922015,49.285714,14.586127,"no const, no equals"
3,LocalOutlierFactor,5,0.3,manhattan,f1 on outer cv inner holdout,66.153846,9.421114,70.392742,10.398666,66.285714,21.094539,"no const, no equals"
4,LocalOutlierFactor,10,0.5,manhattan,accuracy on outer holdout inner cv,62.307692,8.213137,74.662698,7.795923,44.523504,11.004032,"no const, no equals"


In [26]:
def outer_holdout_inner_cv_f1(X):
    n_neighborss = [5, 10, 15]
    contaminations = [0.01, 0.05, 0.1, 0.3, 0.5]
    metrics = ['euclidean', 'minkowski', 'manhattan']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
        best_f1 = 0

        for trainval_idx, valid_idx in cv.split(X_train, y_train):
            X_trainval, X_valid = X_train[trainval_idx], X_train[valid_idx]
            y_trainval, y_valid = y_train[trainval_idx], y_train[valid_idx]

            idxs_neg = np.where(y_trainval == 1)[0]
            X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
            y_valid = np.append(y_valid, y_trainval[idxs_neg])

            X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
            y_trainval = np.delete(y_trainval, idxs_neg)

            for params in itertools.product(n_neighborss, contaminations, metrics):
                clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

                clf.fit(X_trainval)

                pred_values = clf.predict(X_valid)
                true_values = [1 if y == 0 else -1 for y in y_valid]

                f1 = f1_score(true_values, pred_values)
                if f1 > best_f1:
                    best_f1 = f1
                    best_params['n_neighbors'] = params[0]
                    best_params['contamination'] = params[1]
                    best_params['metric'] = params[2]

        clf = LocalOutlierFactor(**best_params, novelty=True)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'f1 on outer holdout inner cv',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [27]:
results = outer_holdout_inner_cv_f1(X)

In [28]:
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,10,0.5,manhattan,accuracy on nested cv,60.153846,8.694690,70.769153,13.875690,45.571429,13.842407,"no const, no equals"
1,LocalOutlierFactor,10,0.3,euclidean,f1 on nested cv,66.769231,9.093856,70.554775,8.376199,65.857143,16.554795,"no const, no equals"
2,LocalOutlierFactor,5,0.5,manhattan,accuracy on outer cv inner holdout,62.000000,10.648777,71.489466,14.922015,49.285714,14.586127,"no const, no equals"
3,LocalOutlierFactor,5,0.3,manhattan,f1 on outer cv inner holdout,66.153846,9.421114,70.392742,10.398666,66.285714,21.094539,"no const, no equals"
4,LocalOutlierFactor,10,0.5,manhattan,accuracy on outer holdout inner cv,62.307692,8.213137,74.662698,7.795923,44.523504,11.004032,"no const, no equals"
5,LocalOutlierFactor,5,0.3,manhattan,f1 on outer holdout inner cv,66.538462,11.416017,73.858364,18.272016,55.719114,20.823273,"no const, no equals"


In [29]:
def double_holdout_accuracy(X):
    n_neighborss = [5, 10, 15]
    contaminations = [0.01, 0.05, 0.1, 0.3, 0.5]
    metrics = ['euclidean', 'minkowski', 'manhattan']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_accuracy = 0

        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(n_neighborss, contaminations, metrics):
            clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            accuracy = accuracy_score(true_values, pred_values)
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params['n_neighbors'] = params[0]
                best_params['contamination'] = params[1]
                best_params['metric'] = params[2]

        clf = LocalOutlierFactor(**best_params, novelty=True)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'accuracy on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [30]:
results = double_holdout_accuracy(X)
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,10,0.5,manhattan,accuracy on nested cv,60.153846,8.694690,70.769153,13.875690,45.571429,13.842407,"no const, no equals"
1,LocalOutlierFactor,10,0.3,euclidean,f1 on nested cv,66.769231,9.093856,70.554775,8.376199,65.857143,16.554795,"no const, no equals"
2,LocalOutlierFactor,5,0.5,manhattan,accuracy on outer cv inner holdout,62.000000,10.648777,71.489466,14.922015,49.285714,14.586127,"no const, no equals"
3,LocalOutlierFactor,5,0.3,manhattan,f1 on outer cv inner holdout,66.153846,9.421114,70.392742,10.398666,66.285714,21.094539,"no const, no equals"
4,LocalOutlierFactor,10,0.5,manhattan,accuracy on outer holdout inner cv,62.307692,8.213137,74.662698,7.795923,44.523504,11.004032,"no const, no equals"
5,LocalOutlierFactor,5,0.3,manhattan,f1 on outer holdout inner cv,66.538462,11.416017,73.858364,18.272016,55.719114,20.823273,"no const, no equals"
6,LocalOutlierFactor,10,0.5,manhattan,accuracy on double holdout,64.230769,14.801447,75.060245,17.584334,51.205840,19.287880,"no const, no equals"


In [31]:
def double_holdout_f1(X):
    n_neighborss = [5, 10, 15]
    contaminations = [0.01, 0.05, 0.1, 0.3, 0.5]
    metrics = ['euclidean', 'minkowski', 'manhattan']
    
    best_params = {'n_neighbors': 0, 'contamination': 0, 'metric': ''}
    accuracy_scores = []
    precision_scores = []
    recall_scores = []

    y = X['Mezzo'].values
    X = X.drop(columns='Mezzo').values

    random_seeds = np.random.randint(2343, 3485327, size=10)
    for seed in random_seeds:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed)

        best_f1 = 0

        X_trainval, X_valid, y_trainval, y_valid = train_test_split(X_train, y_train, test_size=0.2, random_state=seed)

        idxs_neg = np.where(y_trainval == 1)[0]
        X_valid = np.append(X_valid, X_trainval[idxs_neg], axis=0)
        y_valid = np.append(y_valid, y_trainval[idxs_neg])

        X_trainval = np.delete(X_trainval, idxs_neg, axis=0)
        y_trainval = np.delete(y_trainval, idxs_neg)

        for params in itertools.product(n_neighborss, contaminations, metrics):
            clf = LocalOutlierFactor(n_neighbors=params[0], contamination=params[1], metric=params[2], novelty=True)

            clf.fit(X_trainval)

            pred_values = clf.predict(X_valid)
            true_values = [1 if y == 0 else -1 for y in y_valid]

            f1 = f1_score(true_values, pred_values)
            if f1 > best_f1:
                best_f1 = f1
                best_params['n_neighbors'] = params[0]
                best_params['contamination'] = params[1]
                best_params['metric'] = params[2]

        clf = LocalOutlierFactor(**best_params, novelty=True)

        idxs_neg = np.where(y_train == 1)[0]
        X_train = np.delete(X_train, idxs_neg, axis=0)

        clf.fit(X_train)

        pred_values = clf.predict(X_test)
        true_values = [1 if y == 0 else -1 for y in y_test]

        accuracy_scores.append(accuracy_score(true_values, pred_values))
        precision_scores.append(precision_score(true_values, pred_values, zero_division=0.0))
        recall_scores.append(recall_score(true_values, pred_values))

    return {'algorythm': 'LocalOutlierFactor',
            'best n_neighbors': best_params['n_neighbors'],
            'best contamination': best_params['contamination'],
            'best metric': best_params['metric'],
            'method used for model selection': 'f1 on double holdout',
            'accuracy mean': np.mean(accuracy_scores) * 100,
            'accuracy std': np.std(accuracy_scores) * 100,
            'precision mean': np.mean(precision_scores) * 100,
            'precision  std': np.std(precision_scores) * 100,
            'recall mean': np.mean(recall_scores) * 100,
            'recall std': np.std(recall_scores) * 100,
            'data set': 'no const, no equals'}

In [32]:
results = double_holdout_f1(X)
scores_df = add_record(scores_df, results)
scores_df

,algorythm,best n_neighbors,best contamination,best metric,method used for model selection,accuracy mean,accuracy std,precision mean,precision std,recall mean,recall std,data set
0,LocalOutlierFactor,10,0.5,manhattan,accuracy on nested cv,60.153846,8.694690,70.769153,13.875690,45.571429,13.842407,"no const, no equals"
1,LocalOutlierFactor,10,0.3,euclidean,f1 on nested cv,66.769231,9.093856,70.554775,8.376199,65.857143,16.554795,"no const, no equals"
2,LocalOutlierFactor,5,0.5,manhattan,accuracy on outer cv inner holdout,62.000000,10.648777,71.489466,14.922015,49.285714,14.586127,"no const, no equals"
3,LocalOutlierFactor,5,0.3,manhattan,f1 on outer cv inner holdout,66.153846,9.421114,70.392742,10.398666,66.285714,21.094539,"no const, no equals"
4,LocalOutlierFactor,10,0.5,manhattan,accuracy on outer holdout inner cv,62.307692,8.213137,74.662698,7.795923,44.523504,11.004032,"no const, no equals"
5,LocalOutlierFactor,5,0.3,manhattan,f1 on outer holdout inner cv,66.538462,11.416017,73.858364,18.272016,55.719114,20.823273,"no const, no equals"
6,LocalOutlierFactor,10,0.5,manhattan,accuracy on double holdout,64.230769,14.801447,75.060245,17.584334,51.205840,19.287880,"no const, no equals"
7,LocalOutlierFactor,5,0.3,manhattan,f1 on double holdout,64.615385,10.291606,74.169287,12.099093,58.668193,15.039746,"no const, no equals"


In [33]:
file_path = "lof_exp1_df.pickle"

with open(file_path, "wb") as file:
    pickle.dump(scores_df, file)